<a href="https://colab.research.google.com/github/inspire007/Image-Recognition-using-CNN---Cats-and-dogs/blob/main/Image_recognition_using_CNN_Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as kImg
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
print(tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

2.4.1
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
imgs = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
dataset = imgs.flow_from_directory('dataset/training_set', target_size=(64,64), batch_size=32, class_mode='binary')

test_imgs = ImageDataGenerator(rescale=1./255)
test_dataset = test_imgs.flow_from_directory('dataset/test_set', target_size=(64,64), batch_size=32, class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
cnn = Sequential([
    Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[64,64,3]),
    Dropout(0.5),
    MaxPool2D(pool_size=2, strides=2),
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

In [5]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
with tf.device('/GPU:0'):
    cnn.fit(x = dataset, validation_data=test_dataset, epochs=25)

Epoch 1/25
250/250 [==============================] - 87s 333ms/step - loss: 0.7198 - accuracy: 0.5272 - val_loss: 0.6748 - val_accuracy: 0.5950
Epoch 2/25
250/250 [==============================] - 48s 192ms/step - loss: 0.6695 - accuracy: 0.5979 - val_loss: 0.6573 - val_accuracy: 0.6305
Epoch 3/25
250/250 [==============================] - 31s 123ms/step - loss: 0.6439 - accuracy: 0.6229 - val_loss: 0.6341 - val_accuracy: 0.6490
Epoch 4/25
250/250 [==============================] - 28s 111ms/step - loss: 0.6177 - accuracy: 0.6615 - val_loss: 0.5871 - val_accuracy: 0.7125
Epoch 5/25
250/250 [==============================] - 30s 118ms/step - loss: 0.5927 - accuracy: 0.6761 - val_loss: 0.5827 - val_accuracy: 0.7255
Epoch 6/25
250/250 [==============================] - 33s 134ms/step - loss: 0.5572 - accuracy: 0.7032 - val_loss: 0.5409 - val_accuracy: 0.7490
Epoch 7/25
250/250 [==============================] - 33s 131ms/step - loss: 0.5274 - accuracy: 0.7319 - val_loss: 0.5362 - val_ac

In [7]:
cnn.save('cat_dog_model.h5')

In [8]:
#cnn = tf.keras.models.load_model('cat_dog_model.h5')
#cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
#single prediction
def pred_img(path):
    img = kImg.load_img(path, target_size=(64,64))
    img = kImg.img_to_array(img)
    img = img/255
    img = np.expand_dims(img, axis=0)
    pred = cnn.predict(img)
    print(pred)
    if pred[0][0] > 0.6:
        return 'Dog'
    else:
        return 'Cat'

In [21]:
for image in os.listdir('test'):
    path = os.path.join('test', image)
    pred = pred_img(path)
    print(f'File {path}: {pred}')

[[0.00015969]]
File test\cat1.jpg: Cat
[[0.4272664]]
File test\cat2.jpg: Cat
[[0.3814422]]
File test\cat3.jpg: Cat
[[0.57982963]]
File test\cat4.jpg: Cat
[[0.6104696]]
File test\cat5.jpg: Dog
[[0.9993249]]
File test\dog1.jpg: Dog
[[0.9742425]]
File test\dog2.jpg: Dog
[[0.95058227]]
File test\dog3.jpg: Dog
